In [9]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt


In [18]:
data_train = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_train_tech_nonscaled.parquet')

data_test = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/updated_standardization/data_set_test_tech_nonscaled.parquet')

firm_data = pd.read_parquet('/Users/sbjpipers/Desktop/FinalThesisQF/FinalThesisQF/Data/Firm_variables/daily_firm_data_median_new.parquet')

# List of columns to drop
columns_to_drop = ['trading_days_till_exp'] + list(firm_data.columns[2:])
# columns_to_drop = ['trading_days_till_exp']


# Drop columns from datasets if they exist
data_train = data_train.drop(columns=columns_to_drop, errors='ignore')
data_test = data_test.drop(columns=columns_to_drop, errors='ignore')

# Ensure that data_val and data_test have the same column order as data_train
data_test = data_test[data_train.columns]

data_trains = data_train.copy()

data_train = data_trains[data_trains['date'] < '2020-01-01']
data_validate = data_trains[data_trains['date'] >= '2020-01-01']

In [33]:
data_validate

,cp_flag,Ticker,date,moneyness,impl_volatility,T,prev_day_iv,prev2_day_iv,BIDLO,ASKHI,...,gold_price,reces_indi,10Y_RIR,1Y_bond,2Y_bond,OPEN_vix,HIGH_vix,LOW_vix,CLOSE_vix,spread_vix
1881,P,AAPL,2020-01-02,-1.861,0.627576,0.007937,0.316222,0.297076,285.22,292.69000,...,535.741935,0.000000,1.92,1.6348,1.6539,14.47,14.69,13.16,13.26,1.53
1882,P,AAPL,2020-01-02,-1.566,0.589063,0.007937,0.306290,0.326549,285.22,292.69000,...,535.741935,0.000000,1.92,1.6348,1.6539,14.47,14.69,13.16,13.26,1.53
1883,P,AAPL,2020-01-02,-1.275,0.512918,0.007937,0.284871,0.307900,285.22,292.69000,...,535.741935,0.000000,1.92,1.6348,1.6539,14.47,14.69,13.16,13.26,1.53
1884,P,AAPL,2020-01-02,-0.985,0.488522,0.007937,0.265837,0.291318,285.22,292.69000,...,535.741935,0.000000,1.92,1.6348,1.6539,14.47,14.69,13.16,13.26,1.53
1885,P,AAPL,2020-01-02,-0.699,0.429982,0.007937,0.249253,0.279259,285.22,292.69000,...,535.741935,0.000000,1.92,1.6348,1.6539,14.47,14.69,13.16,13.26,1.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145306,C,TSLA,2020-12-31,1.467,0.623205,0.023810,0.554273,0.486818,655.00,669.90002,...,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.82,20.56,21.31,1.26
145307,C,TSLA,2020-12-31,1.535,0.629719,0.023810,0.554273,0.486818,655.00,669.90002,...,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.82,20.56,21.31,1.26
145308,C,TSLA,2020-12-31,1.670,0.639880,0.023810,0.584788,0.511182,655.00,669.90002,...,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.82,20.56,21.31,1.26
145309,C,TSLA,2020-12-31,1.804,0.648805,0.023810,0.601543,0.523462,655.00,669.90002,...,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.82,20.56,21.31,1.26


In [12]:
# import numpy as np
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error, r2_score

# # Assuming 'data_train' is already defined and loaded
# data_train_c = data_train[data_train['cp_flag'] == 'C']
# data_train_p = data_train[data_train['cp_flag'] == 'P']

# def prepare_data(data, option_type):
#     """
#     Prepare the data and train Random Forest for a given dataset.
#     Prints the in-sample RMSE and R².
    
#     Parameters:
#     data (pd.DataFrame): The input dataset.
#     option_type (str): Call or Put option type for labeling the print output.
#     """
#     # Prepare the train data
#     X = data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Features
#     y = data['impl_volatility']  # Target variable
    
#     # Initialize the RandomForestRegressor
#     rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

#     # Train the model
#     rf_model.fit(X, y)

#     # In-sample (training set) predictions
#     y_train_pred = rf_model.predict(X)

#     # Evaluate In-Sample Performance (on Training Data)
#     rmse_train = np.sqrt(mean_squared_error(y, y_train_pred))
#     r2_train = r2_score(y, y_train_pred)

#     # Print the results
#     print(f"\nIn-Sample Performance for {option_type} Options:")
#     print(f"Random Forest RMSE (Training): {rmse_train:.4f}")
#     print(f"Random Forest R² (Training): {r2_train:.4f}")


# # Call the function for Call options data
# prepare_data(data_train_c, 'Call')

# # Call the function for Put options data
# prepare_data(data_train_p, 'Put')


# data_test_c = data_test[data_test['cp_flag'] == 'C']
# data_test_p = data_test[data_test['cp_flag'] == 'P']


# y_test_c = data_test_c['impl_volatility']
# y_test_p = data_test_p['impl_volatility']

# X_test_c = data_test_c.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])
# X_test_p = data_test_p.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])

# # Predict on the test set
# y_pred = rf_model.predict(X_test_c)

# # Evaluate model performance
# rmse_rf = np.sqrt(mean_squared_error(y_test_c, y_pred))
# r2_rf = r2_score(y_test_c, y_pred)

# print(f"Random Forest RMSE: {rmse_rf:.4f}")
# print(f"Random Forest R²: {r2_rf:.4f}")


In [13]:
# import numpy as np
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error, r2_score

# # Assuming 'data_train' and 'data_test' are already defined and loaded

# # Prepare train data for Call and Put options
# data_train_c = data_train[data_train['cp_flag'] == 'C']
# data_train_p = data_train[data_train['cp_flag'] == 'P']

# # Prepare test data for Call and Put options
# data_test_c = data_test[data_test['cp_flag'] == 'C']
# data_test_p = data_test[data_test['cp_flag'] == 'P']

# def prepare_data(train_data, test_data, option_type):
#     """
#     Prepare the data, train Random Forest, and evaluate both in-sample and out-of-sample performance.
    
#     Parameters:
#     train_data (pd.DataFrame): The training dataset.
#     test_data (pd.DataFrame): The testing dataset.
#     option_type (str): Call or Put option type for labeling the print output.
#     """
#     # Prepare the train and test data
#     X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
#     y_train = train_data['impl_volatility']  # Training target
    
#     X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features
#     y_test = test_data['impl_volatility']  # Test target
    
#     # Initialize the RandomForestRegressor
#     rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

#     # Train the model
#     rf_model.fit(X_train, y_train)

#     # In-sample (training set) predictions
#     y_train_pred = rf_model.predict(X_train)

#     # Out-of-sample (test set) predictions
#     y_test_pred = rf_model.predict(X_test)

#     # Evaluate In-Sample Performance (on Training Data)
#     rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
#     r2_train = r2_score(y_train, y_train_pred)

#     # Evaluate Out-of-Sample Performance (on Test Data)
#     rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
#     r2_test = r2_score(y_test, y_test_pred)

#     # Print the results
#     print(f"\nPerformance for {option_type} Options:")
#     print(f"In-Sample RMSE (Training): {rmse_train:.4f}")
#     print(f"In-Sample R² (Training): {r2_train:.4f}")
#     print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
#     print(f"Out-of-Sample R² (Test): {r2_test:.4f}")

# # Call the function for Call options data
# prepare_data(data_train_c, data_test_c, 'Call')

# # Call the function for Put options data
# prepare_data(data_train_p, data_test_p, 'Put')


In [20]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

# Assuming 'data_train' and 'data_test' are already defined and loaded

# Prepare train data for Call and Put options
data_train_c = data_train[data_train['cp_flag'] == 'C']
data_train_p = data_train[data_train['cp_flag'] == 'P']

# Prepare validation data for Call and Put options
data_validate_c = data_validate[data_validate['cp_flag'] == 'C']
data_validate_p = data_validate[data_validate['cp_flag'] == 'P']

# Prepare test data for Call and Put options
data_test_c = data_test[data_test['cp_flag'] == 'C']
data_test_p = data_test[data_test['cp_flag'] == 'P']

In [15]:
data_validate_c

,cp_flag,Ticker,date,moneyness,impl_volatility,T,prev_day_iv,prev2_day_iv,BIDLO,ASKHI,...,gold_price,reces_indi,10Y_RIR,1Y_bond,2Y_bond,OPEN_vix,HIGH_vix,LOW_vix,CLOSE_vix,spread_vix
74856,C,AAPL,2020-01-02,-0.415,0.395077,0.007937,0.219748,0.257932,285.22,292.69000,...,535.741935,0.000000,1.92,1.6348,1.6539,14.47,14.69,13.16,13.26,1.53
74857,C,AAPL,2020-01-02,-0.133,0.342038,0.007937,0.212392,0.253059,285.22,292.69000,...,535.741935,0.000000,1.92,1.6348,1.6539,14.47,14.69,13.16,13.26,1.53
74858,C,AAPL,2020-01-02,0.146,0.320892,0.007937,0.209724,0.251866,285.22,292.69000,...,535.741935,0.000000,1.92,1.6348,1.6539,14.47,14.69,13.16,13.26,1.53
74859,C,AAPL,2020-01-02,0.423,0.306323,0.007937,0.209129,0.253682,285.22,292.69000,...,535.741935,0.000000,1.92,1.6348,1.6539,14.47,14.69,13.16,13.26,1.53
74860,C,AAPL,2020-01-02,0.698,0.294776,0.007937,0.211117,0.259173,285.22,292.69000,...,535.741935,0.000000,1.92,1.6348,1.6539,14.47,14.69,13.16,13.26,1.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145306,C,TSLA,2020-12-31,1.467,0.623205,0.023810,0.554273,0.486818,655.00,669.90002,...,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.82,20.56,21.31,1.26
145307,C,TSLA,2020-12-31,1.535,0.629719,0.023810,0.554273,0.486818,655.00,669.90002,...,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.82,20.56,21.31,1.26
145308,C,TSLA,2020-12-31,1.670,0.639880,0.023810,0.584788,0.511182,655.00,669.90002,...,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.82,20.56,21.31,1.26
145309,C,TSLA,2020-12-31,1.804,0.648805,0.023810,0.601543,0.523462,655.00,669.90002,...,670.825806,3.042903,0.93,0.1105,0.1443,21.50,21.82,20.56,21.31,1.26


In [32]:
# import time
# import itertools
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error, r2_score
# import numpy as np

# def prepare_data_with_gridsearch(train_data, validate_data, test_data, option_type, verbose=True):

#     """
#     Prepare the data, train Random Forest by manually tuning hyperparameters with validation data, 
#     and evaluate both in-sample and out-of-sample performance.
    
#     Parameters:
#     train_data (pd.DataFrame): The training dataset (year 1).
#     validate_data (pd.DataFrame): The validation dataset (year 2).
#     test_data (pd.DataFrame): The testing dataset.
#     option_type (str): Call or Put option type for labeling the print output.
#     verbose (bool): If True, prints progress information for hyperparameter tuning.
#     """
#     # Prepare the train, validation, and test data
#     X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
#     y_train = train_data['impl_volatility']  # Training target
    
#     X_validate = validate_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Validation features
#     y_validate = validate_data['impl_volatility']  # Validation target
    
#     X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features
#     y_test = test_data['impl_volatility']  # Test target
    
#     # Define the hyperparameter grid
#     param_grid = {
#         'max_features': [5, 10, 15],                 # Use fewer features to reduce model complexity
#         'n_estimators': [100, 200, 300],             # Increase number of trees for better generalization
#         'min_samples_leaf': [5, 10, 15],             # Ensure minimum leaf size is larger to reduce overfitting
#         'min_samples_split': [2, 5],                 # Ensure reasonable splits, prevent unnecessary splits
#         'max_depth': [5, 10, 15, 20]                 # Limit the depth of trees to prevent overfitting
#     }
    
#     # Generate all combinations of hyperparameters
#     param_combinations = list(itertools.product(
#         param_grid['max_features'], 
#         param_grid['n_estimators'], 
#         param_grid['min_samples_leaf'], 
#         param_grid['min_samples_split'], 
#         param_grid['max_depth']
#     ))
    
#     total_combinations = len(param_combinations)
    
#     # Initialize variables to store the best model and best score
#     best_rmse_val = np.inf
#     best_params = None
#     best_rf_model = None
    
#     print(f"Running manual hyperparameter tuning for {option_type} Options...")
    
#     # Iterate over all hyperparameter combinations with progress tracking
#     for i, (max_features, n_estimators, min_samples_leaf, min_samples_split, max_depth) in enumerate(param_combinations):
#         start_time = time.time()
        
#         # Initialize the RandomForestRegressor with the current set of hyperparameters
#         rf_model = RandomForestRegressor(
#             max_features=max_features, 
#             n_estimators=n_estimators,
#             min_samples_leaf=min_samples_leaf,
#             min_samples_split=min_samples_split,
#             max_depth=max_depth
#         )
    
#         # Train the model on the training data (Year 1)
#         rf_model.fit(X_train, y_train)
    
#         # Validate the model on the validation data (Year 2)
#         y_val_pred = rf_model.predict(X_validate)
#         rmse_val = np.sqrt(mean_squared_error(y_validate, y_val_pred))
    
#         # Track the best performing hyperparameters based on validation RMSE
#         if rmse_val < best_rmse_val:
#             best_rmse_val = rmse_val
#             best_params = {
#                 'max_features': max_features,
#                 'n_estimators': n_estimators,
#                 'min_samples_leaf': min_samples_leaf,
#                 'min_samples_split': min_samples_split,
#                 'max_depth': max_depth
#             }
#             best_rf_model = rf_model
    
#         # Verbose output to track progress
#         if verbose:
#             elapsed_time = time.time() - start_time
#             print(f"Combination {i + 1}/{total_combinations} completed in {elapsed_time:.2f} seconds.")
#             print(f"Current RMSE (Validation): {rmse_val:.4f}")
#             print(f"Best RMSE so far: {best_rmse_val:.4f}")
    
#     print(f"\nBest Parameters for {option_type} Options: {best_params}")
    
#     # After finding the best hyperparameters, evaluate performance on the test data
#     # In-sample (training set) predictions
#     y_train_pred = best_rf_model.predict(X_train)
    
#     # Validate the predictions
#     y_val_pred = best_rf_model.predict(X_validate)
    
#     # Out-of-sample (test set) predictions
#     y_test_pred = best_rf_model.predict(X_test)
    
#     # Evaluate In-Sample Performance (on Training Data)
#     rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
#     r2_train = r2_score(y_train, y_train_pred)
    
#     # Evaluate Validation Set Performance
#     rmse_val = np.sqrt(mean_squared_error(y_validate, y_val_pred))
#     r2_val = r2_score(y_validate, y_val_pred)
    
#     # Evaluate Out-of-Sample Performance (on Test Data)
#     rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
#     r2_test = r2_score(y_test, y_test_pred)
    
#     # Print the results
#     print(f"\nPerformance for {option_type} Options:")
#     print(f"In-Sample RMSE (Training): {rmse_train:.4f}")
#     print(f"In-Sample R² (Training): {r2_train:.4f}")
#     print(f"Validation RMSE: {rmse_val:.4f}")
#     print(f"Validation R²: {r2_val:.4f}")
#     print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
#     print(f"Out-of-Sample R² (Test): {r2_test:.4f}")

# prepare_data_with_gridsearch(data_train_c, data_validate_c, data_test_c, 'Call')

# prepare_data_with_gridsearch(data_train_p, data_validate_p, data_test_p, 'Put')

In [31]:
import time
import itertools
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

def prepare_data_with_gridsearch(train_data, validate_data, test_data, option_type, verbose=True):
    """
    Prepare the data, perform hyperparameter tuning using Year 1 (train) and Year 2 (validation),
    retrain the model on Year 1 + Year 2, and evaluate on Year 3 (test).
    
    Parameters:
    train_data (pd.DataFrame): The training dataset (Year 1).
    validate_data (pd.DataFrame): The validation dataset (Year 2).
    test_data (pd.DataFrame): The testing dataset (Year 3).
    option_type (str): Call or Put option type for labeling the print output.
    verbose (bool): If True, prints progress information for hyperparameter tuning.
    """
    # Prepare the train, validation, and test data
    X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features (Year 1)
    y_train = train_data['impl_volatility']  # Training target (Year 1)

    X_validate = validate_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Validation features (Year 2)
    y_validate = validate_data['impl_volatility']  # Validation target (Year 2)
    
    X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features (Year 3)
    y_test = test_data['impl_volatility']  # Test target (Year 3)

    # Define the hyperparameter grid
    param_grid = {
        'max_features': [5, 10, 15, 20],                 # Use fewer features to reduce model complexity
        'n_estimators': [50, 100, 200],             # Increase number of trees for better generalization
        'min_samples_leaf': [5, 10, 15],             # Ensure minimum leaf size is larger to reduce overfitting
        'min_samples_split': [2, 5],                 # Ensure reasonable splits, prevent unnecessary splits
        'max_depth': [5, 10, 15, None]                 # Limit the depth of trees to prevent overfitting
    }
    # Best params Call
    # param_grid = {'max_features': [20], 'n_estimators': [50], 'min_samples_leaf': [7], 'min_samples_split': [2], 'max_depth': [None]}

    # Best params Put


    # Generate all combinations of hyperparameters
    param_combinations = list(itertools.product(
        param_grid['max_features'], 
        param_grid['n_estimators'], 
        param_grid['min_samples_leaf'], 
        param_grid['min_samples_split'], 
        param_grid['max_depth']
    ))

    total_combinations = len(param_combinations)
    
    # Initialize variables to store the best model and best score
    best_rmse_val = np.inf
    best_params = None
    best_rf_model = None

    print(f"Running manual hyperparameter tuning for {option_type} Options...")
    
    # Iterate over all hyperparameter combinations with progress tracking
    for i, (max_features, n_estimators, min_samples_leaf, min_samples_split, max_depth) in enumerate(param_combinations):
        start_time = time.time()
        
        # Initialize the RandomForestRegressor with the current set of hyperparameters
        rf_model = RandomForestRegressor(
            max_features=max_features, 
            n_estimators=n_estimators,
            min_samples_leaf=min_samples_leaf,
            min_samples_split=min_samples_split,
            max_depth=max_depth
        )

        # Train the model on the training data (Year 1)
        rf_model.fit(X_train, y_train)

        # Validate the model on the validation data (Year 2)
        y_val_pred = rf_model.predict(X_validate)
        rmse_val = np.sqrt(mean_squared_error(y_validate, y_val_pred))

        # Track the best performing hyperparameters based on validation RMSE
        if rmse_val < best_rmse_val:
            best_rmse_val = rmse_val
            best_params = {
                'max_features': max_features,
                'n_estimators': n_estimators,
                'min_samples_leaf': min_samples_leaf,
                'min_samples_split': min_samples_split,
                'max_depth': max_depth
            }
            best_rf_model = rf_model

        # Verbose output to track progress
        if verbose:
            elapsed_time = time.time() - start_time
            print(f"Combination {i + 1}/{total_combinations} completed in {elapsed_time:.2f} seconds.")
            print(f"Current RMSE (Validation): {rmse_val:.4f}")
            print(f"Best RMSE so far: {best_rmse_val:.4f}")
    
    print(f"\nBest Parameters for {option_type} Options: {best_params}")
    
    # Retrain the model on combined Year 1 (train) and Year 2 (validation)
    print("Retraining the model on Year 1 and Year 2 combined...")
    X_combined = pd.concat([X_train, X_validate])
    y_combined = pd.concat([y_train, y_validate])
    best_rf_model.fit(X_combined, y_combined)

    # In-sample (combined Year 1 + Year 2) predictions
    y_combined_pred = best_rf_model.predict(X_combined)

    # Evaluate In-Sample Performance (on combined Year 1 + Year 2)
    rmse_combined = np.sqrt(mean_squared_error(y_combined, y_combined_pred))
    r2_combined = r2_score(y_combined, y_combined_pred)
    
    print(f"\nIn-Sample Performance for {option_type} Options (Year 1 + Year 2):")
    print(f"RMSE (Training + Validation): {rmse_combined:.4f}")
    print(f"R² (Training + Validation): {r2_combined:.4f}")

    # After retraining, evaluate performance on the test data (Year 3)
    y_test_pred = best_rf_model.predict(X_test)

    # Evaluate Out-of-Sample Performance (on Test Data)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)

    # Print the final results
    print(f"\nPerformance on Test Data (Year 3) for {option_type} Options:")
    print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
    print(f"Out-of-Sample R² (Test): {r2_test:.4f}")


# Call the function for Call options data
prepare_data_with_gridsearch(data_train_c, data_validate_c, data_test_c, 'Call')

# Call the function for Put options data
# prepare_data_with_gridsearch(data_train_p, data_validate_p, data_test_p, 'Put')


Running manual hyperparameter tuning for Call Options...
Combination 1/1 completed in 7.10 seconds.
Current RMSE (Validation): 0.2876
Best RMSE so far: 0.2876

Best Parameters for Call Options: {'max_features': 20, 'n_estimators': 50, 'min_samples_leaf': 7, 'min_samples_split': 2, 'max_depth': None}
Retraining the model on Year 1 and Year 2 combined...

In-Sample Performance for Call Options (Year 1 + Year 2):
RMSE (Training + Validation): 0.0351
R² (Training + Validation): 0.9909

Performance on Test Data (Year 3) for Call Options:
Out-of-Sample RMSE (Test): 0.1325
Out-of-Sample R² (Test): 0.6916


In [5]:
# def prepare_data_with_gridsearch(train_data, validate_data, test_data, option_type):
#     """
#     Prepare the data, train Random Forest using GridSearchCV, and evaluate both in-sample and out-of-sample performance.
    
#     Parameters:
#     train_data (pd.DataFrame): The training dataset.
#     test_data (pd.DataFrame): The testing dataset.
#     option_type (str): Call or Put option type for labeling the print output.
#     """
#     # Prepare the train and test data
#     X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
#     y_train = train_data['impl_volatility']  # Training target

#     # Prepare the validation data
#     X_validate = validate_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
#     y_validate = validate_data['impl_volatility']  # Training target
    
#     X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features
#     y_test = test_data['impl_volatility']  # Test target
    
#     # # Define the parameter grid for GridSearchCV
#     # param_grid = {
#     #     'n_estimators': [100, 200],      # Number of trees in the forest
#     #     'max_depth': [10, 20, None],     # Maximum depth of the tree
#     #     'min_samples_split': [2, 5],     # Minimum number of samples required to split an internal node
#     #     'min_samples_leaf': [1, 2],      # Minimum number of samples required to be at a leaf node
#     #     'bootstrap': [True, False]       # Whether bootstrap samples are used when building trees
#     # }

#     # Hyperparameters Best
#     # param_grid = {
#     #     'max_features': [5,10,15, 20, 25, 30],
#     #     'max_depth': [3,4,5, 6, 7 ,8, 10],
#     #     'n_estimators': [50, 100,150, 200,300, 400, 500, 600, 700],
#     # } np.linspace(0, 1, 10)

#     param_grid = {
#     'max_features': [5, 10, 15],                 # Use fewer features to reduce model complexity
#     'n_estimators': [100, 200, 300],             # Increase number of trees for better generalization
#     'min_samples_leaf': [5, 10, 15],             # Ensure minimum leaf size is larger to reduce overfitting
#     'min_samples_split': [2, 5],                 # Ensure reasonable splits, prevent unnecessary splits
#     'max_depth': [5, 10, 15, 20]                 # Limit the depth of trees to prevent overfitting
#     }   

#     # Initialize the RandomForestRegressor
#     rf_model = RandomForestRegressor()

#     # Initialize GridSearchCV
#     grid_search = GridSearchCV(estimator=rf_model, param_grid=(param_grid), 
#                                cv=2, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

#     # Train the model using GridSearchCV
#     print(f"Running GridSearchCV for {option_type} Options...")
#     grid_search.fit(X_train, y_train)

#     # Get the best parameters and the best estimator
#     best_rf_model = grid_search.best_estimator_
#     print(f"\nBest Parameters for {option_type} Options: {grid_search.best_params_}")

#     # In-sample (training set) predictions
#     y_train_pred = best_rf_model.predict(X_train)

#     # Validate the predictions
#     y_val_pred = best_rf_model.predict(X_validate)

#     # Out-of-sample (test set) predictions
#     y_test_pred = best_rf_model.predict(X_test)

#     # Evaluate In-Sample Performance (on Training Data)
#     rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
#     r2_train = r2_score(y_train, y_train_pred)

#     # Evaluate Validation Set Performance
#     rmse_val = np.sqrt(mean_squared_error(y_validate, y_val_pred))
#     r2_val = r2_score(y_validate, y_val_pred)

#     # Evaluate Out-of-Sample Performance (on Test Data)
#     rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
#     r2_test = r2_score(y_test, y_test_pred)

#     # Print the results
#     print(f"\nPerformance for {option_type} Options:")
#     print(f"In-Sample RMSE (Training): {rmse_train:.4f}")
#     print(f"In-Sample R² (Training): {r2_train:.4f}")
#     print(f"Validation RMSE: {rmse_val:.4f}")
#     print(f"Validation R²: {r2_val:.4f}")
#     print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
#     print(f"Out-of-Sample R² (Test): {r2_test:.4f}")


# # Call the function for Call options data
# prepare_data_with_gridsearch(data_train_c, data_validate_c, data_test_c, 'Call')

# # Call the function for Put options data
# prepare_data_with_gridsearch(data_train_p, data_validate_p, data_test_p, 'Put')


Running GridSearchCV for Call Options...
Fitting 2 folds for each of 216 candidates, totalling 432 fits
[CV] END max_depth=5, max_features=5, min_samples_leaf=5, min_samples_split=2, n_estimators=100; total time=   2.5s
[CV] END max_depth=5, max_features=5, min_samples_leaf=5, min_samples_split=5, n_estimators=100; total time=   2.5s
[CV] END max_depth=5, max_features=5, min_samples_leaf=5, min_samples_split=2, n_estimators=100; total time=   3.1s
[CV] END max_depth=5, max_features=5, min_samples_leaf=5, min_samples_split=5, n_estimators=100; total time=   3.0s
[CV] END max_depth=5, max_features=5, min_samples_leaf=5, min_samples_split=2, n_estimators=200; total time=   5.0s
[CV] END max_depth=5, max_features=5, min_samples_leaf=5, min_samples_split=2, n_estimators=200; total time=   6.2s


KeyboardInterrupt: 

In [9]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

# Assuming 'data_train' and 'data_test' are already defined and loaded

# Prepare train data for Call and Put options
data_train_c = data_train[data_train['cp_flag'] == 'C']
data_train_p = data_train[data_train['cp_flag'] == 'P']

# Prepare test data for Call and Put options
data_test_c = data_test[data_test['cp_flag'] == 'C']
data_test_p = data_test[data_test['cp_flag'] == 'P']

def prepare_data_with_gridsearch(train_data, test_data, option_type):
    """
    Prepare the data, train Random Forest using GridSearchCV, and evaluate both in-sample and out-of-sample performance.
    
    Parameters:
    train_data (pd.DataFrame): The training dataset.
    test_data (pd.DataFrame): The testing dataset.
    option_type (str): Call or Put option type for labeling the print output.
    """
    # Prepare the train and test data
    X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
    y_train = train_data['impl_volatility']  # Training target
    
    X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features
    y_test = test_data['impl_volatility']  # Test target
    
    # # Define the parameter grid for GridSearchCV
    # param_grid = {
    #     'n_estimators': [100, 200],      # Number of trees in the forest
    #     'max_depth': [10, 20, None],     # Maximum depth of the tree
    #     'min_samples_split': [2, 5],     # Minimum number of samples required to split an internal node
    #     'min_samples_leaf': [1, 2],      # Minimum number of samples required to be at a leaf node
    #     'bootstrap': [True, False]       # Whether bootstrap samples are used when building trees
    # }

    # Hyperparameters Best
    # param_grid = {
    #     'max_features': [5,10,15, 20, 25, 30],
    #     'max_depth': [3,4,5, 6, 7 ,8, 10],
    #     'n_estimators': [50, 100,150, 200,300, 400, 500, 600, 700],
    # } np.linspace(0, 1, 10)

    param_grid = {
    'max_features': [5, 10, 15],
    'n_estimators': [100, 200, 300],
    'min_samples_leaf': [10, 20, 30],
    'min_samples_split': [5, 10, 15],
    'max_depth': [5, 10, 15]  # Limit depth to avoid overfitting
    }  





    # Initialize the RandomForestRegressor
    rf_model = RandomForestRegressor(random_state=42, max_samples=0.8)

    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=rf_model, param_grid=(param_grid), 
                               cv=5, n_jobs=-1, verbose=2)

    # Train the model using GridSearchCV
    print(f"Running GridSearchCV for {option_type} Options...")
    grid_search.fit(X_train, y_train)

    # Get the best parameters and the best estimator
    best_rf_model = grid_search.best_estimator_
    print(f"\nBest Parameters for {option_type} Options: {grid_search.best_params_}")

    # In-sample (training set) predictions
    y_train_pred = best_rf_model.predict(X_train)

    # Out-of-sample (test set) predictions
    y_test_pred = best_rf_model.predict(X_test)

    # Evaluate In-Sample Performance (on Training Data)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    r2_train = r2_score(y_train, y_train_pred)

    # Evaluate Out-of-Sample Performance (on Test Data)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)

    # Print the results
    print(f"\nPerformance for {option_type} Options:")
    print(f"In-Sample RMSE (Training): {rmse_train:.4f}")
    print(f"In-Sample R² (Training): {r2_train:.4f}")
    print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
    print(f"Out-of-Sample R² (Test): {r2_test:.4f}")


# Call the function for Call options data
prepare_data_with_gridsearch(data_train_c, data_test_c, 'Call')

# Call the function for Put options data
prepare_data_with_gridsearch(data_train_p, data_test_p, 'Put')


Running GridSearchCV for Call Options...
Fitting 5 folds for each of 243 candidates, totalling 1215 fits
[CV] END max_depth=5, max_features=5, min_samples_leaf=10, min_samples_split=5, n_estimators=100; total time=   5.2s
[CV] END max_depth=5, max_features=5, min_samples_leaf=10, min_samples_split=5, n_estimators=100; total time=   5.6s
[CV] END max_depth=5, max_features=5, min_samples_leaf=10, min_samples_split=5, n_estimators=100; total time=   5.8s
[CV] END max_depth=5, max_features=5, min_samples_leaf=10, min_samples_split=5, n_estimators=100; total time=   6.1s
[CV] END max_depth=5, max_features=5, min_samples_leaf=10, min_samples_split=5, n_estimators=100; total time=   6.0s
[CV] END max_depth=5, max_features=5, min_samples_leaf=10, min_samples_split=5, n_estimators=200; total time=  10.2s
[CV] END max_depth=5, max_features=5, min_samples_leaf=10, min_samples_split=5, n_estimators=200; total time=  10.5s
[CV] END max_depth=5, max_features=5, min_samples_leaf=10, min_samples_split

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

# Assuming 'data_train' and 'data_test' are already defined and loaded

# Prepare train data for Call and Put options
data_train_c = data_train[data_train['cp_flag'] == 'C']
data_train_p = data_train[data_train['cp_flag'] == 'P']

# Prepare test data for Call and Put options
data_test_c = data_test[data_test['cp_flag'] == 'C']
data_test_p = data_test[data_test['cp_flag'] == 'P']

def prepare_data_with_gridsearch(train_data, test_data, option_type):
    """
    Prepare the data, train Random Forest using GridSearchCV, and evaluate both in-sample and out-of-sample performance.
    Additionally, select the best models with in-sample R² closest to 0.80 and 0.85.
    
    Parameters:
    train_data (pd.DataFrame): The training dataset.
    test_data (pd.DataFrame): The testing dataset.
    option_type (str): Call or Put option type for labeling the print output.
    """
    # Prepare the train and test data
    X_train = train_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Training features
    y_train = train_data['impl_volatility']  # Training target
    
    X_test = test_data.drop(columns=['impl_volatility', 'date', 'Ticker', 'cp_flag'])  # Test features
    y_test = test_data['impl_volatility']  # Test target
    
    # Define the parameter grid for GridSearchCV
    param_grid = {
        'max_features': [5, 10, 15],                 # Use fewer features to reduce model complexity
        'n_estimators': [100, 200, 300],             # Increase number of trees for better generalization
        'min_samples_leaf': [5, 10, 15],             # Ensure minimum leaf size is larger to reduce overfitting
        'min_samples_split': [2, 5],                 # Ensure reasonable splits, prevent unnecessary splits
        'max_depth': [5, 10, 15, 20]                 # Limit the depth of trees to prevent overfitting
    }    

    # Initialize the RandomForestRegressor
    rf_model = RandomForestRegressor()

    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, 
                               cv=2, n_jobs=-1, verbose=2)

    # Train the model using GridSearchCV
    print(f"Running GridSearchCV for {option_type} Options...")
    grid_search.fit(X_train, y_train)

    # Initialize tracking variables for the models closest to r² = 0.80 and r² = 0.85
    closest_to_80 = {'model': None, 'params': None, 'r2_train': float('inf')}
    closest_to_85 = {'model': None, 'params': None, 'r2_train': float('inf')}

    # Iterate over the results and select the best models with in-sample R² closest to 0.80 and 0.85
    for idx, params in enumerate(grid_search.cv_results_['params']):
        # Get the estimator with the current parameters
        model = grid_search.estimator.set_params(**params)
        model.fit(X_train, y_train)

        # In-sample (training set) predictions
        y_train_pred = model.predict(X_train)
        r2_train = r2_score(y_train, y_train_pred)

        # Track the model closest to r² = 0.80
        if abs(r2_train - 0.80) < abs(closest_to_80['r2_train'] - 0.80):
            closest_to_80['model'] = model
            closest_to_80['params'] = params
            closest_to_80['r2_train'] = r2_train
        
        # Track the model closest to r² = 0.85
        if abs(r2_train - 0.85) < abs(closest_to_85['r2_train'] - 0.85):
            closest_to_85['model'] = model
            closest_to_85['params'] = params
            closest_to_85['r2_train'] = r2_train

    # Evaluate models closest to r² = 0.80 and r² = 0.85
    for target_r2, model_info in zip([0.80, 0.85], [closest_to_80, closest_to_85]):
        if model_info['model'] is None:
            print(f"\nNo suitable model found for {option_type} Options with in-sample R² closest to {target_r2}.")
            continue
        
        # Print the best parameters
        print(f"\nBest Parameters for {option_type} Options (R² closest to {target_r2}): {model_info['params']}")
        print(f"In-Sample R² (Training): {model_info['r2_train']:.4f}")

        # Out-of-sample (test set) predictions
        y_test_pred = model_info['model'].predict(X_test)

        # Evaluate Out-of-Sample Performance (on Test Data)
        rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
        r2_test = r2_score(y_test, y_test_pred)

        # Print the results
        print(f"Out-of-Sample RMSE (Test): {rmse_test:.4f}")
        print(f"Out-of-Sample R² (Test): {r2_test:.4f}")


# Call the function for Call options data
prepare_data_with_gridsearch(data_train_c, data_test_c, 'Call')

# Call the function for Put options data
prepare_data_with_gridsearch(data_train_p, data_test_p, 'Put')

print("hello")
